In [ ]:
import os
import numpy as np
import pandas as pd
from PIL import Image
from datetime import datetime
from flask import Flask, request, jsonify
from flask_cors import CORS
from tensorflow.keras.models import load_model

SIMPLE_MODEL_PATH = "C:/Users/ABA/Downloads/Xception.h5"
DETAILED_MODEL_PATH ="C:/Users/ABA/Desktop/New folder (3)/detailed_ex.h5"
CSV_PATH = "E:/me/my Projects/dataset/csv/modified_dataset.csv"

if not os.path.exists(SIMPLE_MODEL_PATH):
    raise FileNotFoundError(f"Simple model not found at: {SIMPLE_MODEL_PATH}")
if not os.path.exists(DETAILED_MODEL_PATH):
    raise FileNotFoundError(f"Detailed model not found at: {DETAILED_MODEL_PATH}")
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"CSV file not found at: {CSV_PATH}")

simple_model = load_model(SIMPLE_MODEL_PATH)
detailed_model = load_model(DETAILED_MODEL_PATH)
csv_data = pd.read_csv(CSV_PATH)

print("Both models and the CSV file were loaded successfully!")

app = Flask(__name__)
CORS(app)

CATEGORIES = ["Cancer", "Non-Cancer"]

def preprocess_image(img, IMG_SIZE):
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)
    return img

def decode_prediction(encoded_value, column_name):
    unique_values = csv_data[column_name].unique()
    return str(unique_values[int(encoded_value)])

@app.route("/")
def home():
    return "Flask API is running successfully! - Use /predict/simple or /predict/detailed"

@app.route("/predict/simple", methods=["POST"])
def predict_simple():
    if "file" not in request.files:
        return jsonify({"error": "No image file uploaded"}), 400

    file = request.files["file"]
    img = Image.open(file).convert("RGB")
    img = preprocess_image(img, 100)

    prediction = simple_model.predict(img)
    predicted_class = np.argmax(prediction, axis=1)[0]
    confidence = float(np.max(prediction))

    return jsonify({
        "class": CATEGORIES[predicted_class],
        "confidence": confidence,
        "file": file.filename
    })

@app.route("/predict/detailed", methods=["POST"])
def predict_detailed():
    if "file" not in request.files:
        return jsonify({"error": "No image file uploaded"}), 400

    file = request.files["file"]
    img = Image.open(file).convert("RGB")
    img = preprocess_image(img, 224)

    predictions = detailed_model.predict(img)

    results = {
        "mass_shape": decode_prediction(np.argmax(predictions[0], axis=1)[0], 'mass_shape'),
        "mass_margins": decode_prediction(np.argmax(predictions[1], axis=1)[0], 'mass_margins'),
        "calc_type": decode_prediction(np.argmax(predictions[2], axis=1)[0], 'calc_type'),
        "calc_distribution": decode_prediction(np.argmax(predictions[3], axis=1)[0], 'calc_distribution'),
        "pathology": decode_prediction(np.argmax(predictions[4], axis=1)[0], 'pathology'),
        "breast_density": decode_prediction(np.argmax(predictions[5], axis=1)[0], 'breast_density'),
        "left_or_right_breast": decode_prediction(np.argmax(predictions[6], axis=1)[0], 'left_or_right_breast'),
        "image_view": decode_prediction(np.argmax(predictions[7], axis=1)[0], 'image_view'),
        "abnormality_id": decode_prediction(np.argmax(predictions[8], axis=1)[0], 'abnormality_id'),
        "abnormality_type": decode_prediction(np.argmax(predictions[9], axis=1)[0], 'abnormality_type')
    }

    report = {
        "report_date": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "processed_image": file.filename,
        "predictions": results,
        "Notes": "Based on these findings, it is advisable to see a specialist to assess the condition accurately."
    }

    return jsonify(report)

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)


Both models and the CSV file were loaded successfully!
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.9:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


INFO:werkzeug:192.168.1.11 - - [12/Jun/2025 20:18:41] "POST /predict/simple HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step


INFO:werkzeug:192.168.1.11 - - [12/Jun/2025 20:18:49] "POST /predict/detailed HTTP/1.1" 200 -
